In [5]:
import pandas as pd
import ast
from funciones import extract
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
%load_ext memory_profiler
import pyarrow.parquet as pq


In [6]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

In [4]:
item = pd.read_csv(r'data_set_limpio/items_preparado.csv.gz')

In [7]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [5]:
def developer(des):
    if des not in list(df['developer']):
        return 'El desarrollador no se encuentra en la base de datos'
    

    fechas = df['release_date'].unique()
    
    anio = {}
    
    free = {}
    for x in fechas:
        if len(df[(df['release_date'] == x) & (df['developer'] == des)]) != 0:
            anio[x] = len(df[(df['release_date'] == x) & (df['developer'] == des)])
            free[x] = len(df[(df['release_date'] == x) & (df['developer'] == des) & (df['price'] == 0.0)])
    for x,y  in free.items():
        free[x] = str(round((y / anio[x])*100,2))+'%'

    resultado = {'Cantidad de Items': anio,'Contenido Free':free}
    
    return resultado




In [8]:

def developer(developer: str):
    if developer not in list(df['developer']):
        return 'El desarrollador no se encuentra en la base de datos'

    fechas = df['release_date'].unique()

    anio = {}
    free = {}

    for x in fechas:
        filter_condition = (df['release_date'] == x) & (df['developer'] == developer)
        developer_releases = df[filter_condition]

        if len(developer_releases) != 0:
            anio[x] = len(developer_releases)
            free[x] = len(developer_releases[developer_releases['price'] == 0.0])

    for x, y in free.items():
        free[x] = f"{round((y / anio[x]) * 100, 2)}%"

    # Convertir a tipos de datos nativos de Python
    anio = {str(k): v for k, v in anio.items()}
    free = {str(k): v for k, v in free.items()}

    resultado = {'Cantidad de Items': anio, 'Contenido Free': free}

    # Serializar a JSON usando json.dumps
    return json.dumps(resultado)


In [9]:
developer('Valve')

'{"Cantidad de Items": {"2017": 2, "1998": 1, "2006": 2, "2003": 1, "2007": 3, "2000": 2, "2001": 1, "2004": 5, "2008": 1, "2009": 1, "1999": 1, "2010": 2, "2011": 1, "2012": 1}, "Contenido Free": {"2017": "0.0%", "1998": "0.0%", "2006": "0.0%", "2003": "0.0%", "2007": "33.33%", "2000": "0.0%", "2001": "0.0%", "2004": "0.0%", "2008": "0.0%", "2009": "0.0%", "1999": "0.0%", "2010": "50.0%", "2011": "0.0%", "2012": "0.0%"}}'

In [7]:
def userdata(user: str):
    
    try:
        for x in pd.read_csv(r'data_set_limpio/items_preparado.csv.gz', chunksize=3000):
            if user in list(x['user_id']):
                aux = x
                break
            x = 0
        
        
        
        precios = []
        respuesta = {}
        
        
        usuario = aux.loc[aux['user_id'] == user]['items']
        
        
        if not usuario.empty:
            usuario = usuario.iloc[0]
        data = ast.literal_eval(usuario)
        result = pd.DataFrame(data)
        result.dropna(inplace=True)
        

        for y in result['item_name']:
            price = df.loc[df['app_name'] == y]['price'].values
            if len(price) > 0:
                try: 
                    price_value = float(price[0])
                    precios.append(price_value)
                except ValueError:
                    pass  
                
        respuesta['Usuario'] = user
        respuesta['Dinero gastado'] = str(round(sum(precios)))+' USD'
        respuesta["cantidad de items"] = str(len(result))
        
        respuesta['Porcentaje de recomendaciones'] = str(round(((len(opinion.loc[opinion['user'] == user]) / len(result))) * 100,2))+ '%'
        
        return respuesta
    except:
        return 'El usuario no se encuentra en la base de datos.'

In [9]:

userdata('76561197970982479')


{'Usuario': '76561197970982479',
 'Dinero gastado': '2315 USD',
 'cantidad de items': '277',
 'Porcentaje de recomendaciones': '1.08%'}

In [12]:
def best_developer_year(year : int):
    
        anio = pd.read_csv(r'data_set_limpio//Max_developer_year.csv')

            
        anio = anio.sort_values('Anio', ascending=False)
        
        seleccion = anio.loc[anio['Anio'] == year].copy()
        
        seleccion.fillna('Sin informacion',inplace=True)
        
        if not seleccion['Anio'].empty:
            respuesta = {'Anio': str(seleccion['Anio'].values[0]),
                        'Top 1': str(seleccion['Top 1'].values[0]),
                        'Top 2': str(seleccion['Top 2'].values[0]),
                        'Top 3': str(seleccion['Top 3'].values[0])
                        }
        else:
            respuesta = {f'No ingreso un valor relevante, este es el rango disponible ({str(anio["Anio"].min())} - {str(anio["Anio"].max())})'}

        return respuesta

In [19]:
best_developer_year(2017)

{'Anio': '2017',
 'Top 1': "('Smartly Dressed Games', 594)",
 'Top 2': "('Freejam', 178)",
 'Top 3': "('Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC', 160)"}

In [20]:
def resenias_developer( des : str ):
    
    try:

        table = pq.read_table(r'data_set_limpio/recomends_dev.parquet')
        
        dev = table.to_pandas()
        
        developer = dev.loc[dev['Developers'] == des]
        
        respuesta = {'Deloper' : str(developer['Developers'].values[0]), 'Reviews positivos' : str(developer['Positivo'].values[0]),'Reviews negativos' : str(developer['Negativo'].values[0])}
        dev = 0
        return respuesta
    
    except:
        
        return 'No ingreso un valor relevante, o el desarrollador no se encuentra en la base de datos'   

In [22]:
resenias_developer('Freejam')

{'Deloper': 'Freejam', 'Reviews positivos': '211', 'Reviews negativos': '71'}